In [25]:
import pandas as pd

In [26]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
!pip install pyarrow


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
# Load January 2023 data
df_jan = pd.read_parquet("yellow_tripdata_2023-02.parquet")

In [29]:
print(f"Number of columns: {len(df_jan.columns)}")

Number of columns: 19


In [30]:
df_jan['tpep_pickup_datetime'] = pd.to_datetime(df_jan['tpep_pickup_datetime'])
df_jan['tpep_dropoff_datetime'] = pd.to_datetime(df_jan['tpep_dropoff_datetime'])


In [31]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60


In [32]:
std_dev = df_jan['duration'].std()
print(f"Standard deviation: {std_dev:.2f}")


Standard deviation: 42.84


In [33]:
df_filtered = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)]


In [34]:
fraction_remaining = len(df_filtered) / len(df_jan)
print(f"Fraction remaining: {fraction_remaining:.2%}")


Fraction remaining: 98.01%


In [35]:
from sklearn.feature_extraction import DictVectorizer

In [36]:
# Recast IDs to string
df_filtered['PULocationID'] = df_filtered['PULocationID'].astype(str)
df_filtered['DOLocationID'] = df_filtered['DOLocationID'].astype(str)

C:\Users\a_farnoo\AppData\Local\Temp\ipykernel_14520\1781947651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['PULocationID'] = df_filtered['PULocationID'].astype(str)
C:\Users\a_farnoo\AppData\Local\Temp\ipykernel_14520\1781947651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['DOLocationID'] = df_filtered['DOLocationID'].astype(str)


In [37]:
# Convert to list of dictionaries
dicts = df_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')


In [38]:
# One-hot encode using DictVectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(dicts)


In [39]:
print(f"Number of features: {X_train.shape[1]}")

Number of features: 514


In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [41]:
y_train = df_filtered['duration']

In [42]:
# Train model
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [43]:
# Predict on training set
y_pred = model.predict(X_train)


In [44]:
# Compute RMSE
rmse_train = mean_squared_error(y_train, y_pred, squared=False)
print(f"Train RMSE: {rmse_train:.2f}")

Train RMSE: 7.78


C:\Users\a_farnoo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
